In [30]:
#-*- coding: EUC-KR -*-

%matplotlib inline 

import numpy as np 
import pandas as pd
import skimage.io as io 
import matplotlib.pyplot as plt 
import pylab 
import json
from tqdm import tqdm
import glob
import os
import seaborn as sns
from pathlib import Path
import base64
import cv2
import scipy 
import time
from joblib import Parallel , delayed
from sklearn.model_selection import train_test_split
from collections import Counter
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
rootpath = "D:/Yolov5_DeepSort_Pytorch/inference/output"


# rootpath = "D:\F1soft 119"

In [31]:
tracking_name_list = [] 

for (path, dir, files) in os.walk(rootpath):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt':
            full=path+'/'+filename
            tracking_name_list.append(full)

In [32]:
tracking_name_list

['D:/Yolov5_DeepSort_Pytorch/inference/output/G025045_000_0_0_3.txt',
 'D:/Yolov5_DeepSort_Pytorch/inference/output/T000020_000_1_2_4.txt',
 'D:/Yolov5_DeepSort_Pytorch/inference/output/T000021_000_1_2_4.txt',
 'D:/Yolov5_DeepSort_Pytorch/inference/output/T000023_000_1_1_3.txt',
 'D:/Yolov5_DeepSort_Pytorch/inference/output/T000024_000_1_1_3.txt',
 'D:/Yolov5_DeepSort_Pytorch/inference/output/T000025_000_1_2_3.txt',
 'D:/Yolov5_DeepSort_Pytorch/inference/output/T000026_000_1_1_3.txt',
 'D:/Yolov5_DeepSort_Pytorch/inference/output/T000027_000_1_1_3.txt',
 'D:/Yolov5_DeepSort_Pytorch/inference/output/T000028_000_1_1_3.txt',
 'D:/Yolov5_DeepSort_Pytorch/inference/output/T000029_000_1_1_3.txt',
 'D:/Yolov5_DeepSort_Pytorch/inference/output/T000030_000_1_1_3.txt',
 'D:/Yolov5_DeepSort_Pytorch/inference/output/T000031_000_1_1_3.txt',
 'D:/Yolov5_DeepSort_Pytorch/inference/output/T000032_000_1_1_3.txt',
 'D:/Yolov5_DeepSort_Pytorch/inference/output/T000033_000_1_1_3.txt',
 'D:/Yolov5_DeepSort

In [33]:
tracking_labels_list=[]

for t_name in tracking_name_list:
    with open(t_name, 'r') as file:
        tracking_labels_list.append(file.read())

In [34]:
print(tracking_labels_list[0])

133 10 3 646 535 63 63 
135 10 3 644 536 59 62 
136 10 3 640 537 59 59 
137 10 3 637 538 60 58 
138 10 3 635 538 59 57 
139 10 3 633 540 57 56 



In [35]:
def main_object(tracking_txt):
    t_length=(len(tracking_txt.split('\n')))
    a=tracking_txt.split('\n')[int(t_length/10):(-int(t_length/10)-1)] # 앞뒤 프레임의 10% 제거 
    line_ap=[]
    unique_id=[]
    
    for num in a:             # line_ap에 object별로 저장
        b=num.split(' ')
        unique_id.append(b[2])
        line_ap.append(b)
        
    unique_id=set(unique_id)   #tracking_id 중복 제거
    
    checking_object =[]  #[tracking_id, 오브젝트 개수, cate_id, 움직임여부(1,0)]
    # 주인공 가능 cate_id : 10~15 
    
    for uq in unique_id:
        ob_line_list=[]
        cate_id=[]
        uq_num=0
        x_min=2000;x_max=-1;y_min=2000;y_max=-1;
        moving_check=0
        
        for line in line_ap:
            if(line[2]==uq):
                uq_num+=1                      #오브젝트 개수
                cate_id.append(line[1])          #cate_id 저장
                
                if(x_min>int(line[3])):          #움직임 여부 확인
                    x_min=int(line[3])
                if(x_max<int(line[3])):
                    x_max=int(line[3])
                if(y_min>int(line[4])):
                    y_min=int(line[4])
                if(y_max<int(line[4])):
                    y_max=int(line[4])
        if(abs(x_min-x_max)>100):
            moving_check=1
        if(abs(y_min-y_max)>100):
            moving_check=1
            
        count_id=Counter(cate_id)                #많은 비중을 차지하는 cate_id
        max_id=count_id.most_common(1)[0][0]     
        
        ob_line_list.append(str(uq))            #[tracking_id, 오브젝트 개수, cate_id, 움직임여부(1,0)]
        ob_line_list.append(uq_num)
        ob_line_list.append(max_id)
        ob_line_list.append(moving_check)
        checking_object.append(ob_line_list)   
        
    return checking_object

In [36]:
def select_mo(data):
    main_list=[str(x) for x in range(10,16)]
    ob_count=0
    cate_id='12'
    for line in data:
        if (line[3]==1) and (line[2] in main_list):
            if ob_count<line[1]:
                ob_count=line[1]
                cate_id=line[2]
    
    return [ob_count,cate_id]
    

In [37]:
print(main_object(tracking_labels_list[15]))

[['188', 16, '12', 0], ['191', 8, '15', 0], ['186', 37, '15', 1], ['187', 45, '10', 0], ['183', 307, '12', 1], ['197', 15, '10', 0], ['196', 14, '11', 0]]


In [237]:
predict=[]

for i in range(len(tracking_labels_list)):
    temp=select_mo(main_object(tracking_labels_list[i]))[1]
    if temp in ['10']:
        predict.append('0')
    elif temp in['11','13']:
        predict.append('1')
    elif temp in['12','14','15']:
        predict.append('2')
#     else:
#         predict.append('3')


In [238]:
answer=[]

In [239]:
for i in tracking_name_list:
    answer.append(((i.split('/')[-1]).split('.')[0]).split('_')[3])

In [240]:
from sklearn.metrics import classification_report

print(classification_report(answer, predict, target_names=['motor', 'bicycle','kick']))

              precision    recall  f1-score   support

       motor       1.00      0.77      0.87        13
     bicycle       0.99      0.83      0.90        81
        kick       0.85      1.00      0.92        93

    accuracy                           0.91       187
   macro avg       0.95      0.87      0.90       187
weighted avg       0.92      0.91      0.91       187



In [241]:
# for i in range(len(answer)):
#     if answer[i]!=predict[i]:
#         print(predict[i],tracking_name_list[i]) 

In [164]:
tracking_name_list[15]

'D:/Yolov5_DeepSort_Pytorch/inference/output/T000043_000_1_2_3.txt'

In [ ]:
#0~1 : 보행자녹색신호,보행자적색신호
#2~4 : 차량녹색신호,차량황색신호,차량적색신호
#5~6 : 차량직좌신호,차량좌신호
#7~9 : 주차된 오토바이,자전거,킥보드

#10~12 : 오토바이탑승자,자전거탑승자,킥보드탑승자 ->킥보드탑승자 오버피팅
#13~14 : 자전거캐리어,킥보드캐리어 -> 언더피팅 이슈 
#15 : 킥보드 동승자 -> 오버피팅 이슈


# 0. 공통사항

#### - 대략적으로 탐지해도 되는 경우 vs 자세하게 탐지해야 하는 경우를 잘 나누어 설정
#### - 정지되어있는 사진으로 도출할 수 있는 위반 사항인지 확인
#### - 필수로 식별해야하는 요소 확인


### 기타 고려사항

###### - 프레임 단위로 labels, images 저장 / 경로 직접 생성 후 저장
###### - 1초당 1프레임 인지 (해당 경우일 때 고려해야하는 사항이 무엇인지?)

In [4]:
objectpath = "D:/Yolov5_DeepSort_Pytorch/inference/output"
roadpath = "D:/Yolov5_DeepSort_Pytorch/inference/output"

tracking_object_list = []
tracking_road_list = [] 

for (path, dir, files) in os.walk(objectpath):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt':
            full=path+'/'+filename
            tracking_object_list.append(full)
            
for (path, dir, files) in os.walk(roadpath):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt':
            full=path+'/'+filename
            tracking_road_list.append(full)

In [18]:
def yolotobbox(list):
    x,y,w,h = [list[i] for i in range(4)]
    x1= x-w/2
    x2=x+w/2
    y1=y-h/2
    y2=y+h/2
    
    return [[x1,y1],[x2,y1],[x2,y2],[x1,y2]]

In [29]:
from shapely.geometry import Polygon

def check_intersection(polygon_coor,bbox_coor):
    polygon1 = Polygon(polygon_coor)
    bbox1 = Polygon(yolotobbox(bbox_coor))

    intersect = polygon1.intersection(polygon2).area
    mainarea = polygon2.area
    iom = intersect / mainarea
    
    return iom


# 1. 보행자도로 통행위반

###### 필수 식별요소(인도,자전거 도로, 차종)
###### 움직임 필요(인도에 주차되어 있는 객체 제외)
###### 대략적으로 탐지해도 됨

In [ ]:
#? 수치들은 pipeline로 성능 좋은걸로 ??
# check_intersection/ bbox는 차종(오자킥, 움직이는 객체만), polygon은 인도 / 수치가 OOO 이상이면 위반
# 프레임 20장이라고 가정할 때 순간위반, 지속위반 중 어느 케이스 ? / 순간 위반이라면 오탐지일때를 대비하여 어느정도 비중까지?

# 2. 동승자 탑승 위반

###### 필수 식별요소(킥보드 동승자)
###### 움직임 필요 X
###### 정확한 탐지가 요구됨
###### crop된 객체를 한번 더 사람 수를 구하는 모델(crowd counting 활용?)
###### 학습한 데이터는 킥보드만이므로 킥보드일 때만 모델 구동

# 3. 안전모 미착용 위반

###### 필수 식별요소(차종, 안전모)
###### 움직임 필요 X
###### 정확한 탐지가 요구됨
###### Annotation을 직접 해야하는 소요 발생 가능성 多

# 4. 무단횡단 위반

###### 필수 식별요소(중앙선, 차종)
###### 움직임 필요(인도에 주차되어 있는 객체 제외)
###### 대략적으로 탐지해도 됨


In [38]:
# 차종 오브젝트의 중심점과 중앙선 폴리곤과 가장 가까운 점의 거리로 판단? 
# + check_intersection 겹치는 영역까지?
# 중앙선 기준 위아래 영역을 생성후 판단

# 5. 횡단보도 주행 위반

###### 필수 식별요소(자전거도로, 횡단보도, 차종, 차종_carrier)
###### 움직임 필요 X
###### 정확한 탐지가 요구됨

In [ ]:
# check_intersection/ bbox는 차종(오자킥, 움직이는 객체만), polygon은 횡단보도 / 수치가 OOO 이상이면 위반
# 횡단보도 옆 자전거도로를 달릴때 / 차종_carrier일 때에 대한 정확한 객체 탐지가 요구됨
#자전거도로와 차종_carrier 식별 잘해야함!

# 6. 신호 위반

###### 필수 식별요소(보행자 신호, 차량 신호,  횡단보도, 차종)
###### 움직임 필요(인도에 주차되어 있는 객체, 기다리고 있는 차종 제외)
######  대략적으로 탐지해도 되지만, 신호에 대한 탐지는 정확해야함

In [39]:
# 보행자도로 빨간불 신호가 존재할때
# check_intersection/ bbox는 차종(오자킥, 움직이는 객체만), polygon은 횡단보도 / 수치가 OOO 이상이면 위반

# 오토바이가 차도에서 위반하는 경우 고려는 어떻게 ???
# 극단적인 경우 오토바이 버린다


# 학습용 데이터에 생각보다 킥보드와 자전거가 차량 신호를 위반하는 경우가 많음
# case 1 : car_road와 main_object의 check_intersection이 높은 상태를 오래 유지할 때
#def isRedLight_V(directory, main_object(tracking_labels_list)):  #directory : 한영상에 대한 프레임단위 좌표가 적혀있는 경로,
                                                                  #tracking detection list
    #for frame in directory:
#         isvehiclelist.append(frame마다 check_intersection(car_road,main_object) 계산)
        
        
    #if (~~~~~~~~) : # check_intersection이
    #if (~~~~~~~):#cate_id : '4'(차량 빨간불)이 일정프레임 이상이라면
#         if(~~~~~~~): #main_object가 움직인다면
#                if check_intersection(횡단보도, 메인차종)이 일정수준(수치는 단계적으로 확인) 이상이라면
#                  return 위반


# case 2 : crosswalk와 main_object의 check_intersection이 높은 상태를 오래 유지할 때
#def isRedLight_P(main_object(tracking_labels_list)):  # tracking detection list 

# 8. 정지선 위반

###### 필수 식별요소(차량 신호,  정지선, 오토바이)
###### 움직임 필요 (?)
###### 대략적으로 탐지해도 되지만, 신호에 대한 탐지는 정확해야함
###### 학습데이터가 오토바이만 있으므로 오토바이일때만 모델 구동

# 영상길이 및 어노테이션 프레임 일치 확인

In [4]:
json_data_list=[]
mp4_fps_list=[]
mp4_framecnt_list=[]

In [5]:
for anno in tqdm(train_json_list): # annotation.json 파일 읽어들이기
    with open(anno, encoding='utf-8') as json_file:
        json_data_list.append(json.load(json_file))


100%|████████████████████████████████████████████████████████████████████████████████| 355/355 [00:39<00:00,  9.00it/s]


In [6]:
for vid_info in tqdm(train_mp4_list): # annotation.mp4 파일 읽어들이기
    cap = cv2.VideoCapture(vid_info) 
    codec = cv2.VideoWriter_fourcc(*'XVID')
    vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    vid_fps = cap.get(cv2.CAP_PROP_FPS )
    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))    
    mp4_fps_list.append(vid_fps)
    mp4_framecnt_list.append(frame_cnt) 

100%|████████████████████████████████████████████████████████████████████████████████| 355/355 [00:26<00:00, 13.59it/s]


In [7]:
frame_check_df=pd.DataFrame(columns=['vid','annotation'])

frame_check_df=pd.DataFrame()

for i in range(len(mp4_framecnt_list)):
    made_row=pd.DataFrame([mp4_framecnt_list[i],len(json_data_list[i]['annotations']['anno_info'])]).T
    frame_check_df=pd.concat([frame_check_df,made_row],axis=0)
    
frame_check_df.reset_index(drop=True,inplace=True)
frame_check_df.rename(columns={0:'vid',1:'annotation'},inplace= True)

In [8]:
frame_check_df['nomatch'] = frame_check_df.apply(lambda x: x['annotation']-x['vid'],axis=1)

In [9]:
frame_check_df['json'] = train_json_list

In [10]:
frame_check_df['mp4'] = train_mp4_list

In [11]:
frame_check_df

,vid,annotation,nomatch,json,mp4
0,512,513,1,D:/F1soft 120해커톤\1 보행자도로 통행 위반\1 자전거\0 정상/T001...,D:/F1soft 120해커톤\1 보행자도로 통행 위반\1 자전거\0 정상/T001...
1,378,379,1,D:/F1soft 120해커톤\1 보행자도로 통행 위반\1 자전거\0 정상/T001...,D:/F1soft 120해커톤\1 보행자도로 통행 위반\1 자전거\0 정상/T001...
2,502,503,1,D:/F1soft 120해커톤\1 보행자도로 통행 위반\1 자전거\0 정상/T001...,D:/F1soft 120해커톤\1 보행자도로 통행 위반\1 자전거\0 정상/T001...
3,427,428,1,D:/F1soft 120해커톤\1 보행자도로 통행 위반\1 자전거\0 정상/T001...,D:/F1soft 120해커톤\1 보행자도로 통행 위반\1 자전거\0 정상/T001...
4,480,481,1,D:/F1soft 120해커톤\1 보행자도로 통행 위반\1 자전거\0 정상/T001...,D:/F1soft 120해커톤\1 보행자도로 통행 위반\1 자전거\0 정상/T001...
...,...,...,...,...,...
350,302,303,1,D:/F1soft 120해커톤\8 정지선 위반\0 오토바이\1 위반/T004244_...,D:/F1soft 120해커톤\8 정지선 위반\0 오토바이\1 위반/T004244_...
351,272,273,1,D:/F1soft 120해커톤\8 정지선 위반\0 오토바이\1 위반/T004244_...,D:/F1soft 120해커톤\8 정지선 위반\0 오토바이\1 위반/T004244_...
352,301,0,-301,D:/F1soft 120해커톤\8 정지선 위반\0 오토바이\1 위반/T004244_...,D:/F1soft 120해커톤\8 정지선 위반\0 오토바이\1 위반/T004244_...
353,302,0,-302,D:/F1soft 120해커톤\8 정지선 위반\0 오토바이\1 위반/T004244_...,D:/F1soft 120해커톤\8 정지선 위반\0 오토바이\1 위반/T004244_...


In [12]:
good_fps=frame_check_df[(frame_check_df['nomatch']<=10) & (frame_check_df['nomatch']>=0)] # 0,5,6

In [13]:
train_json_list=good_fps['json'].to_list()
train_mp4_list=good_fps['mp4'].to_list()

In [14]:
train_json_list.remove('D:/F1soft 120해커톤\\2 동승자 탑승 위반\\2 킥보드\\1 위반/T002628_000_1_2_2.json')
train_mp4_list.remove('D:/F1soft 120해커톤\\2 동승자 탑승 위반\\2 킥보드\\1 위반/T002628_000_1_2_2.mp4')

# train_test_split

In [15]:
train_json_data_list=[]
test_json_data_list=[]


In [16]:
train_mp4_list_split, test_mp4_list_split = train_test_split(train_mp4_list, test_size=0.2, random_state=0)

In [17]:
train_json_list_split, test_json_list_split = train_test_split(train_json_list, test_size=0.2, random_state=0)

In [18]:
train_mp4_list_split

['D:/F1soft 120해커톤\\5 횡단보도 주행 위반\\1 자전거\\1 위반/T001847_000_1_1_5.mp4',
 'D:/F1soft 120해커톤\\5 횡단보도 주행 위반\\1 자전거\\1 위반/T001811_000_1_1_5.mp4',
 'D:/F1soft 120해커톤\\3 안전모 미착용 위반\\0 오토바이\\0 정상/T003997_017_0_0_3.mp4',
 'D:/F1soft 120해커톤\\2 동승자 탑승 위반\\2 킥보드\\1 위반/T002485_000_1_2_2.mp4',
 'D:/F1soft 120해커톤\\6 신호 위반\\2 킥보드\\1 위반/T001877_000_1_2_6.mp4',
 'D:/F1soft 120해커톤\\8 정지선 위반\\0 오토바이\\1 위반/T003998_064_1_0_8.mp4',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\1 위반/T001904_000_1_1_1.mp4',
 'D:/F1soft 120해커톤\\6 신호 위반\\2 킥보드\\1 위반/T002056_000_1_2_6.mp4',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\1 위반/T001851_000_1_1_1.mp4',
 'D:/F1soft 120해커톤\\5 횡단보도 주행 위반\\1 자전거\\0 정상/T001875_000_0_1_5.mp4',
 'D:/F1soft 120해커톤\\3 안전모 미착용 위반\\0 오토바이\\0 정상/T003997_013_0_0_3.mp4',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\0 정상/T001892_000_0_1_1.mp4',
 'D:/F1soft 120해커톤\\3 안전모 미착용 위반\\1 자전거\\1 위반/T001840_000_1_1_3.mp4',
 'D:/F1soft 120해커톤\\6 신호 위반\\1 자전거\\0 정상/T001908_000_0_1_6.mp4',
 'D:/F1soft 120해커톤\\8 정지선 위반\\0 오토

In [24]:
sorted(train_json_list_split)

['D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\0 정상/T001783_000_0_1_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\0 정상/T001815_000_0_1_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\0 정상/T001817_000_0_1_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\0 정상/T001822_000_0_1_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\0 정상/T001834_000_0_1_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\0 정상/T001872_000_0_1_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\0 정상/T001892_000_0_1_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\0 정상/T001930_000_0_1_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\1 위반/T001785_000_1_1_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\1 위반/T001802_000_1_1_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\1 위반/T001824_000_1_1_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\1 위반/T001836_000_1_1_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\1 위반/T001851_000_1_1_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\1 위반/T001871_000_1_1_1

In [23]:
sorted(test_json_list_split)

['D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\0 정상/T001853_000_0_1_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\0 정상/T001905_000_0_1_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\1 위반/T001813_000_1_1_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\2 킥보드\\0 정상/T001823_000_0_2_1.json',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\2 킥보드\\1 위반/T001906_000_1_2_1.json',
 'D:/F1soft 120해커톤\\2 동승자 탑승 위반\\2 킥보드\\1 위반/T001854_000_1_2_2.json',
 'D:/F1soft 120해커톤\\2 동승자 탑승 위반\\2 킥보드\\1 위반/T001971_000_1_2_2.json',
 'D:/F1soft 120해커톤\\2 동승자 탑승 위반\\2 킥보드\\1 위반/T002161_000_1_2_2.json',
 'D:/F1soft 120해커톤\\2 동승자 탑승 위반\\2 킥보드\\1 위반/T002549_000_1_2_2.json',
 'D:/F1soft 120해커톤\\2 동승자 탑승 위반\\2 킥보드\\1 위반/T002812_000_1_2_2.json',
 'D:/F1soft 120해커톤\\3 안전모 미착용 위반\\0 오토바이\\0 정상/T003997_005_0_0_3.json',
 'D:/F1soft 120해커톤\\3 안전모 미착용 위반\\0 오토바이\\0 정상/T003997_011_0_0_3.json',
 'D:/F1soft 120해커톤\\3 안전모 미착용 위반\\1 자전거\\0 정상/T001886_000_0_1_3.json',
 'D:/F1soft 120해커톤\\3 안전모 미착용 위반\\1 자전거\\1 위반/T002511_000_1_1_3.json',
 'D:

In [25]:
for anno in tqdm(train_json_list_split): # annotation.json 파일 읽어들이기
    with open(anno, encoding='utf-8') as json_file:
        train_json_data_list.append(json.load(json_file))

100%|████████████████████████████████████████████████████████████████████████████████| 203/203 [00:16<00:00, 12.54it/s]


In [26]:
for anno in tqdm(test_json_list_split): # annotation.json 파일 읽어들이기
    with open(anno, encoding='utf-8') as json_file:
        test_json_data_list.append(json.load(json_file))

100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [00:09<00:00,  5.45it/s]


In [48]:
def imwrite(filename, img, params=None):
    try: 
        ext = os.path.splitext(filename)[1]
        result, n = cv2.imencode(ext, img, params)
        if result:
            with open(filename, mode='w+b') as f:
                n.tofile(f)
            return True
        else:
            return False
    except Exception as e:
        print(e)
        return False

In [49]:
def id_transform(category_id):
    a=1

    ca_id=[2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 17, 18, 19, 20, 21, 22, 24, 25, 27, 30, 31, 33, 43, 45, 46,
           47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 60, 61]
    
    ca_viol_id=[62, 64, 66, 69, 70, 73, 74, 75, 76, 78, 115, 118, 123, 124, 125, 126, 127, 131, 132]
    
    viol_allo=[0, 34, 2,  7,  0,  2,  3,  4,  5,  7,  24,  27,  32,  33,  34,  35,  38,39,39] #38:킥보드동승자탑승_위반, 39:자전거표지판_06_표지판
    
    for id, idx in zip(ca_id,range(len(ca_id))):
        if category_id==str(id):
            return idx
        else:
            a=40
            
    for id, idx in zip(ca_viol_id,viol_allo):
        if category_id==str(id):
            return idx
        else:
            a=40
            
    if a==40:
        return 40

In [50]:
def id_transform_road(category_id):
    a=1

    ca_id=[2, 3, 4, 5, 6, 7, 8, 9]
    
    ca_viol_id=[62, 66, 69, 70, 73, 74, 75, 76, 78]
    
    viol_allo=[0, 2,  7,  0,  2,  3,  4,  5,  7] 
    
    for id, idx in zip(ca_id,range(len(ca_id))):
        if category_id==str(id):
            return idx
        else:
            a=8
            
    for id, idx in zip(ca_viol_id,viol_allo):
        if category_id==str(id):
            return idx
        else:
            a=8
            
    if a==8:
        return 8

In [51]:
def id_transform_object(category_id):
    a=1

#     ca_id=[11, 13, 17, 18, 19, 20, 21, 22, 24, 25, 27, 30, 31, 33, 43, 45, 46,
#            47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 60, 61] # 0~29

    ca_id=[45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 60, 61] # 0~14
    
#     ca_viol_id=[64, 115, 118, 123, 124, 125, 126, 127, 131, 132]
#     viol_allo=[26, 16,  19,  24,  25,  26,  27,  30, 31 , 31] #38:킥보드동승자탑승_위반, 39:자전거표지판_06_표지판
    
    ca_viol_id=[64, 115, 118, 123, 124, 125, 126, 127]
    
    viol_allo=[11, 1,  4,  12,  10,  11,  12,  15] #15:킥보드동승자탑승_위반

    for id, idx in zip(ca_id,range(len(ca_id))):
        if category_id==str(id):
            return idx
        else:
            a=16
            
    for id, idx in zip(ca_viol_id,viol_allo):
        if category_id==str(id):
            return idx
        else:
            a=16
            
    if a==16:
        return 16

In [52]:
def bbox(anno_info):
    bbox = []

    for obj in anno_info:
        coor = obj['annotation_info']
        if len(coor)==0:
            continue
        elif len(coor) !=1:
#             print(len(coor))
            xyxy = bounding_box(coor)
        else:
            xyxy = coor
            
        b_center_x = (xyxy[0][0] + xyxy[0][2]) / 2
        b_center_y = (xyxy[0][1] + xyxy[0][3]) / 2
        b_width    = xyxy[0][2] - xyxy[0][0]
        b_height   = xyxy[0][3] - xyxy[0][1]
        
        b_center_x/=1920
        b_center_y/=1080
        b_width/=1920
        b_height/=1080
        ca_id=id_transform(obj['category_id'])
        bbox.append("{} {:.6f} {:.6f} {:.6f} {:.6f}".format(ca_id, b_center_x, b_center_y, b_width, b_height))
        
    return bbox

In [53]:
def bbox_object(anno_info):
    bbox = []

    for obj in anno_info:
        if ((int(obj['category_id'])>=45) and (int(obj['category_id'])<=61)) or obj['category_id'] in ['64', '115', '118', '123', '124', '125', '126', '127'] :
            coor = obj['annotation_info']
            if len(coor)==0:
                continue
            elif len(coor) !=1:
    #             print(len(coor))
                xyxy = bounding_box(coor)
            else:
                xyxy = coor

            b_center_x = (xyxy[0][0] + xyxy[0][2]) / 2
            b_center_y = (xyxy[0][1] + xyxy[0][3]) / 2
            b_width    = xyxy[0][2] - xyxy[0][0]
            b_height   = xyxy[0][3] - xyxy[0][1]

            b_center_x/=1920
            b_center_y/=1080
            b_width/=1920
            b_height/=1080
            ca_id=id_transform_object(obj['category_id'])
            bbox.append("{} {:.6f} {:.6f} {:.6f} {:.6f}".format(ca_id, b_center_x, b_center_y, b_width, b_height))
        
    return bbox

In [54]:
def bbox_road(anno_info):
    bbox = []

    for obj in anno_info:
        if (int(obj['category_id'])<10) or (obj['category_id'] in ['62', '66', '69', '70', '73', '74', '75', '76', '78']) :
            coor = obj['annotation_info']
            if len(coor)==0:
                continue
            elif len(coor) !=1:
    #             print(len(coor))
                xyxy = bounding_box(coor)
            else:
                xyxy = coor

            b_center_x = (xyxy[0][0] + xyxy[0][2]) / 2
            b_center_y = (xyxy[0][1] + xyxy[0][3]) / 2
            b_width    = xyxy[0][2] - xyxy[0][0]
            b_height   = xyxy[0][3] - xyxy[0][1]

            b_center_x/=1920
            b_center_y/=1080
            b_width/=1920
            b_height/=1080
            ca_id=id_transform_road(obj['category_id'])

            bbox.append("{} {:.6f} {:.6f} {:.6f} {:.6f}".format(ca_id, b_center_x, b_center_y, b_width, b_height))
        
    return bbox

In [55]:
def bounding_box(points):
    x_coordinates, y_coordinates = zip(*points)
    
    return [[min(x_coordinates), min(y_coordinates), max(x_coordinates), max(y_coordinates)]]

In [56]:
import cv2


output_path = 'D:/'
# video_output_path = 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/'

In [57]:
train_mp4_list_image=[]

train_mp4_list_labels_road=[]
train_mp4_list_labels_object=[]

for i in train_mp4_list_split:
    train_mp4_list_image.append(output_path+'images_t/train/'+(i.split('/',)[-1]).split('.')[0])
    train_mp4_list_labels_road.append(output_path+'labels_road/train/'+(i.split('/',)[-1]).split('.')[0])   
    train_mp4_list_labels_object.append(output_path+'labels_object/train/'+(i.split('/',)[-1]).split('.')[0])                             

In [58]:
test_mp4_list_image=[]
test_mp4_list_labels_road=[]
test_mp4_list_labels_object=[]


for i in test_mp4_list_split:
    test_mp4_list_image.append(output_path+'images_t/valid/'+(i.split('/',)[-1]).split('.')[0])
    test_mp4_list_labels_road.append(output_path+'labels_road/valid/'+(i.split('/',)[-1]).split('.')[0])
    test_mp4_list_labels_object.append(output_path+'labels_object/valid/'+(i.split('/',)[-1]).split('.')[0])                             

In [59]:
test_real_mp4_list=[]

In [60]:
train_mp4_list_image

['D:/images_t/train/T001847_000_1_1_5',
 'D:/images_t/train/T001811_000_1_1_5',
 'D:/images_t/train/T003997_017_0_0_3',
 'D:/images_t/train/T002485_000_1_2_2',
 'D:/images_t/train/T001877_000_1_2_6',
 'D:/images_t/train/T003998_064_1_0_8',
 'D:/images_t/train/T001904_000_1_1_1',
 'D:/images_t/train/T002056_000_1_2_6',
 'D:/images_t/train/T001851_000_1_1_1',
 'D:/images_t/train/T001875_000_0_1_5',
 'D:/images_t/train/T003997_013_0_0_3',
 'D:/images_t/train/T001892_000_0_1_1',
 'D:/images_t/train/T001840_000_1_1_3',
 'D:/images_t/train/T001908_000_0_1_6',
 'D:/images_t/train/T003999_044_1_0_8',
 'D:/images_t/train/T001835_000_1_2_1',
 'D:/images_t/train/T001978_000_1_1_4',
 'D:/images_t/train/T001918_000_0_1_5',
 'D:/images_t/train/T004244_034_0_0_8',
 'D:/images_t/train/T003999_006_1_0_8',
 'D:/images_t/train/T001867_000_1_2_6',
 'D:/images_t/train/T002825_000_1_1_3',
 'D:/images_t/train/T001871_000_1_1_1',
 'D:/images_t/train/T003997_012_1_1_3',
 'D:/images_t/train/T003997_016_0_0_3',


In [61]:
train_mp4_list_split

['D:/F1soft 120해커톤\\5 횡단보도 주행 위반\\1 자전거\\1 위반/T001847_000_1_1_5.mp4',
 'D:/F1soft 120해커톤\\5 횡단보도 주행 위반\\1 자전거\\1 위반/T001811_000_1_1_5.mp4',
 'D:/F1soft 120해커톤\\3 안전모 미착용 위반\\0 오토바이\\0 정상/T003997_017_0_0_3.mp4',
 'D:/F1soft 120해커톤\\2 동승자 탑승 위반\\2 킥보드\\1 위반/T002485_000_1_2_2.mp4',
 'D:/F1soft 120해커톤\\6 신호 위반\\2 킥보드\\1 위반/T001877_000_1_2_6.mp4',
 'D:/F1soft 120해커톤\\8 정지선 위반\\0 오토바이\\1 위반/T003998_064_1_0_8.mp4',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\1 위반/T001904_000_1_1_1.mp4',
 'D:/F1soft 120해커톤\\6 신호 위반\\2 킥보드\\1 위반/T002056_000_1_2_6.mp4',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\1 위반/T001851_000_1_1_1.mp4',
 'D:/F1soft 120해커톤\\5 횡단보도 주행 위반\\1 자전거\\0 정상/T001875_000_0_1_5.mp4',
 'D:/F1soft 120해커톤\\3 안전모 미착용 위반\\0 오토바이\\0 정상/T003997_013_0_0_3.mp4',
 'D:/F1soft 120해커톤\\1 보행자도로 통행 위반\\1 자전거\\0 정상/T001892_000_0_1_1.mp4',
 'D:/F1soft 120해커톤\\3 안전모 미착용 위반\\1 자전거\\1 위반/T001840_000_1_1_3.mp4',
 'D:/F1soft 120해커톤\\6 신호 위반\\1 자전거\\0 정상/T001908_000_0_1_6.mp4',
 'D:/F1soft 120해커톤\\8 정지선 위반\\0 오토

# FPS, Size

In [62]:
import time
import imutils

green_color=(0, 255, 0)
red_color=(0, 0, 255)
train_real_mp4_list=[]

for v_input, i_output,la_output1,la_output2, k in tqdm(zip(train_mp4_list_split,train_mp4_list_image,train_mp4_list_labels_road,train_mp4_list_labels_object,range(len(train_mp4_list_split)))):
#     if k==1:
#         break
    
    frame_rate=10
    prev =0
    num=1
    cap = cv2.VideoCapture(v_input) # <- 앞으로 이 변수를 중심으로 사용할 것
    # Codec은 *'XVID'로 설정. 
    codec = cv2.VideoWriter_fourcc(*'XVID')  # 코덱 객체를 생성
#     vid_size1 = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))) #(1920, 1080)   640 360
#     cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)                                    
    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))    

    hasFrame, img_frame = cap.read()
    
    for i in range(1,frame_cnt):
        time_elapsed = time.time() - prev
        hasFrame, img_frame = cap.read()
#         img_frame = imutils.resize(img_frame, width=640, height = 360)

        if time_elapsed > 1./frame_rate:
            prev = time.time()
            train_real_mp4_list.append(str(i_output+'_{:03}'.format(i)))
            imwrite(i_output+'_{:03}'.format(num)+'.jpg',img_frame)
#             print("\n".join(bbox_road(train_json_data_list[k]['annotations']['anno_info'][i]['frame_annotation'])), file= open(la_output1+'_{:03}'.format(num)+'.txt', "w"))
            print("\n".join(bbox_object(train_json_data_list[k]['annotations']['anno_info'][i]['frame_annotation'])), file= open(la_output2+'_{:03}'.format(num)+'.txt', "w"))

            num+=1

    cap.release()         # 처음에 가져온 이미지. file.close() 와 같은 느낌

203it [11:32,  3.41s/it]


In [63]:
test_real_mp4_list=[]
1111
for v_input, i_output,la_output1,la_output2, k in tqdm(zip(test_mp4_list_split,test_mp4_list_image,test_mp4_list_labels_road,test_mp4_list_labels_object,range(len(test_mp4_list_split)))):
#     if k==1:
#         break
    
    frame_rate=10
    prev =0
    num=1
    cap = cv2.VideoCapture(v_input) # <- 앞으로 이 변수를 중심으로 사용할 것
    # Codec은 *'XVID'로 설정. 
    codec = cv2.VideoWriter_fourcc(*'XVID')  # 코덱 객체를 생성
#     vid_size1 = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))) #(1920, 1080)
#     cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)   
    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))    

    hasFrame, img_frame = cap.read()
    
    for i in range(1,frame_cnt):
        time_elapsed = time.time() - prev
        hasFrame, img_frame = cap.read()
#         img_frame = imutils.resize(img_frame, width=640, height = 360)
    

        if time_elapsed > 1./frame_rate:
            prev = time.time()
            test_real_mp4_list.append(str(i_output+'_{:03}'.format(i)))
            imwrite(i_output+'_{:03}'.format(num)+'.jpg',img_frame)
#             print("\n".join(bbox_road(test_json_data_list[k]['annotations']['anno_info'][i]['frame_annotation'])), file= open(la_output1+'_{:03}'.format(num)+'.txt', "w"))
            print("\n".join(bbox_object(test_json_data_list[k]['annotations']['anno_info'][i]['frame_annotation'])), file= open(la_output2+'_{:03}'.format(num)+'.txt', "w"))

            num+=1

    cap.release()         # 처음에 가져온 이미지. file.close() 와 같은 느낌

51it [03:36,  4.24s/it]


In [44]:
test_mp4_list_labels_object

['D:/labels_road/valid/T001796_000_1_1_5',
 'D:/labels_road/valid/T001886_000_0_1_3',
 'D:/labels_road/valid/T001839_000_0_2_5',
 'D:/labels_road/valid/T002830_000_0_2_3',
 'D:/labels_road/valid/T001858_000_0_1_5',
 'D:/labels_road/valid/T002053_000_1_1_6',
 'D:/labels_road/valid/T002551_000_1_1_4',
 'D:/labels_road/valid/T003997_068_1_0_8',
 'D:/labels_road/valid/T002549_000_1_2_2',
 'D:/labels_road/valid/T002506_000_1_2_4',
 'D:/labels_road/valid/T001845_000_0_1_5',
 'D:/labels_road/valid/T003998_057_1_0_8',
 'D:/labels_road/valid/T001983_000_1_1_4',
 'D:/labels_road/valid/T001808_000_1_1_6',
 'D:/labels_road/valid/T001882_000_0_1_5',
 'D:/labels_road/valid/T003997_011_0_0_3',
 'D:/labels_road/valid/T002055_000_0_2_6',
 'D:/labels_road/valid/T001849_000_0_2_5',
 'D:/labels_road/valid/T001792_000_1_1_5',
 'D:/labels_road/valid/T001798_000_0_2_3',
 'D:/labels_road/valid/T001797_000_1_1_6',
 'D:/labels_road/valid/T001971_000_1_2_2',
 'D:/labels_road/valid/T003998_082_1_0_8',
 'D:/labels

In [43]:
train_real_mp4_list

['D:/images_t/train/T001847_000_1_1_5_001',
 'D:/images_t/train/T001847_000_1_1_5_005',
 'D:/images_t/train/T001847_000_1_1_5_022',
 'D:/images_t/train/T001847_000_1_1_5_039',
 'D:/images_t/train/T001847_000_1_1_5_056',
 'D:/images_t/train/T001847_000_1_1_5_069',
 'D:/images_t/train/T001847_000_1_1_5_085',
 'D:/images_t/train/T001847_000_1_1_5_101',
 'D:/images_t/train/T001847_000_1_1_5_113',
 'D:/images_t/train/T001847_000_1_1_5_123',
 'D:/images_t/train/T001847_000_1_1_5_137',
 'D:/images_t/train/T001847_000_1_1_5_153',
 'D:/images_t/train/T001847_000_1_1_5_168',
 'D:/images_t/train/T001847_000_1_1_5_185',
 'D:/images_t/train/T001847_000_1_1_5_198',
 'D:/images_t/train/T001847_000_1_1_5_208',
 'D:/images_t/train/T001847_000_1_1_5_220',
 'D:/images_t/train/T001847_000_1_1_5_233',
 'D:/images_t/train/T001847_000_1_1_5_247',
 'D:/images_t/train/T001847_000_1_1_5_262',
 'D:/images_t/train/T001847_000_1_1_5_273',
 'D:/images_t/train/T001847_000_1_1_5_288',
 'D:/images_t/train/T001847_000_

In [ ]:
frame_rate = 10
prev = 0

while capturing:

    time_elapsed = time.time() - prev
    res, image = cap.read()

    if time_elapsed > 1./frame_rate:
        prev = time.time()

        # Do something with your image here.
        process_image()

In [3]:
def xyxy2coco(anno_info):
    coor=[]
    for i in range(0,4):
        coor.append(anno_info[0][i])
    w,h =  coor[2]-coor[0], coor[3]-coor[1]
    return (int(coor[0]),int(coor[1]),int(w),int(h))

In [4]:
def caption_bbox(anno_info):
    coor=[]
    for i in range(0,4):
        coor.append(anno_info[0][i])
#     w,h =  coor[2]-coor[0], coor[3]-coor[1]
    return (int(coor[0]),int(coor[1]))

# Yolo format Transform

In [5]:
def xyxy2coco(xyxy):
    x1,y1,x2,y2 =xyxy
    w,h =  x2-x1, y2-y1
    return [x1,y1,w,h] 

def xyxy2yolo(xyxy):
    
    x1,y1,x2,y2 =xyxy
    w,h =  x2-x1, y2-y1
    xc = x1 + int(np.round(w/2)) # xmin + width/2
    yc = y1 + int(np.round(h/2)) # ymin + height/2
    return [xc/IMG_SIZE,yc/IMG_SIZE,w/IMG_SIZE,h/IMG_SIZE] 

def scale_bbox(img, xyxy):
    # Get scaling factor
    scale_x = IMG_SIZE/img.shape[1]
    scale_y = IMG_SIZE/img.shape[0]
    
    x1,y1,x2,y2 =xyxy
    x1 = int(np.round(x1*scale_x, 4))
    y1 = int(np.round(y1*scale_y, 4))
    x2 = int(np.round(x2*scale_x, 4))
    y2= int(np.round(y2*scale_y, 4))

    return [x1, y1, x2, y2] # xmin, ymin, xmax, ymax

def save_image_label(json_file,mode): 
    with open(json_file,'r') as f: 
        json_file =json.load(f)

    image_id = json_file['file_name'].replace('.json','')
    
    # decode image data
    image = np.frombuffer(base64.b64decode(json_file['imageData']), np.uint8)
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    cv2.imwrite(str(new_image_path / (image_id + '.png')) ,image)
    
    # extract bbox
    origin_bbox = []
    if mode == 'train':
        with open(new_label_path / (image_id + '.txt'), 'w') as f:
            for i in json_file['shapes']: 
                bbox = i['points'][0] + i['points'][2]
                origin_bbox.append(bbox)
                bbox = scale_bbox(image,bbox)
                bbox = xyxy2yolo(bbox)
                
                labels = [categories[i['label']]]+bbox
                f.writelines([f'{i} ' for i in labels] + ['\n']) 
    return origin_bbox

# json 불러오기

In [6]:
train_json_list = [] 

train_json_list
for (path, dir, files) in os.walk(rootpath):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.json':
            full=path+'/'+filename
            train_json_list.append(full)

In [7]:
train_mp4_list = [] 

for (path, dir, files) in os.walk(rootpath):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.mp4':
            full=path+'/'+filename
            train_mp4_list.append(full)

In [8]:
del train_json_list[0]
del train_json_list[4:6]

In [9]:
del train_mp4_list[0]
del train_mp4_list[4:6]


IndexError: list assignment index out of range

In [15]:
json_data_list=[]

In [16]:
for anno in tqdm(train_json_list): # annotation.json 파일 읽어들이기
    with open(anno, encoding='utf-8') as json_file:
        json_data_list.append(json.load(json_file))


100%|████████████████████████████████████████████████████████████████████████████████| 186/186 [00:26<00:00,  6.99it/s]


# 영상처리 관련

In [17]:
from IPython.display import clear_output, Image, display, Video, HTML

In [6]:

# # cap = cv2.VideoCapture(0)   # 0: default camera
# cap = cv2.VideoCapture(rootpath + "/1 보행자도로 통행 위반/오토바이/위반/T004935_048_1_0_1.mp4") #동영상 파일에서 읽기
 
# while cap.isOpened():
#     # 카메라 프레임 읽기
#     success, frame = cap.read()
#     if success:
#         # 프레임 출력
#         cv2.imshow('Camera Window', frame)
 
#         # ESC를 누르면 종료
#         key = cv2.waitKey(1) & 0xFF
#         if (key == 27): 
#             break

            
# cap.release()
# cv2.destroyAllWindows()

In [6]:
import cv2

video_input_path = rootpath + "/1 보행자도로 통행 위반/오토바이/위반/T004936_006_1_0_1.mp4"
# linux에서 video output의 확장자는 반드시 avi 로 설정 필요. 
video_output_path = 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/test/'
# video_output_path = 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/'


In [8]:

cap = cv2.VideoCapture(video_input_path) # <- 앞으로 이 변수를 중심으로 사용할 것
# Codec은 *'XVID'로 설정. 
codec = cv2.VideoWriter_fourcc(*'XVID')  # 코덱 객체를 생성
vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))) #(1920, 1080)
vid_fps = cap.get(cv2.CAP_PROP_FPS ) 
frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))    

print(vid_fps)
vid_writer = cv2.VideoWriter(video_output_path, codec, vid_fps, vid_size) # 처음에는 아무것도 없는 동영상
print('총 Frame 갯수:', frame_cnt, 'FPS:', round(vid_fps), 'Frame 크기:', vid_size)

29.97
총 Frame 갯수: 181 FPS: 30 Frame 크기: (1920, 1080)


In [74]:
import time

green_color=(0, 255, 0)
red_color=(0, 0, 255)

start = time.time()
index=0

while True:
    hasFrame, img_frame = cap.read() 
    # cap.read가 frame 하나하나를 순서대로 뱉어준다. 한 프래임씩 읽어주는 아주 좋은 매소드함수라고 할 수 있다.
    if not hasFrame:
        print('더 이상 처리할 frame이 없습니다.')
        break
    index += 1
    print('frame :', index, '처리 완료')
    cv2.rectangle(img_frame, (300, 100, 800, 400), color=green_color, thickness=2)  # 동영상 프래임 이미지 각각에 박스 넣기
    caption = "frame:{}".format(index)
    cv2.putText(img_frame, caption, (300, 95), cv2.FONT_HERSHEY_SIMPLEX, 0.7, red_color, 1)  # 동영상 프레임 이미지 각각에 몇번째 Frame인지 적기
    
    vid_writer.write(img_frame) 
    # 동영상에 차곡차곡 동영상프래임이미지를 쌓아올린다. 

print('write 완료 시간:', round(time.time()-start,4))
vid_writer.release()  # 나중에 저장한 동영상. file.close() 와 같은 느낌
cap.release()         # 처음에 가져온 이미지. file.close() 와 같은 느낌

더 이상 처리할 frame이 없습니다.
write 완료 시간: 0.001


In [69]:
train_mp4_list

['C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/RAW데이터/대용량데이터/PM 사고위험 영상데이터/1 보행자도로 통행 위반\\오토바이\\위반/T004936_006_1_0_1.mp4',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/RAW데이터/대용량데이터/PM 사고위험 영상데이터/1 보행자도로 통행 위반\\오토바이\\위반/T004936_020_1_0_1.mp4',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/RAW데이터/대용량데이터/PM 사고위험 영상데이터/1 보행자도로 통행 위반\\오토바이\\위반/T004936_022_1_0_1.mp4',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/RAW데이터/대용량데이터/PM 사고위험 영상데이터/1 보행자도로 통행 위반\\오토바이\\위반/T004936_033_1_0_1.mp4',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/RAW데이터/대용량데이터/PM 사고위험 영상데이터/1 보행자도로 통행 위반\\자전거\\위반/T000087_000_1_1_1.mp4',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/RAW데이터/대용량데이터/PM 사고위험 영상데이터/1 보행자도로 통행 위반\\자전거\\위반/T000088_000_1_1_1.mp4',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/RAW데이터/대용량데이터/PM 사고위험 영상데이터/1 보행자도로 통행 위반\\자전거\\위반/T000089_000_1_1_1.mp4',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/RAW데이터/대용량데이터/PM 사고위험 영상데이터/1 보행자도로 통행 위반\\자전거\\위반/T000090_000_1_1_1.mp4',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/RAW데이터/대용량데이터/PM 사고위험 영상데이터/1 보행자도로 통행 위반\\자전

In [71]:
train_mp4_list_output=[]
train_mp4_list_image=[]
train_mp4_list_labels=[]

for i in train_mp4_list:
    train_mp4_list_output.append(video_output_path+'output_'+i.split('/',)[-1])
    train_mp4_list_image.append(video_output_path+'image/'+(i.split('/',)[-1]).split('.')[0])
    train_mp4_list_labels.append(video_output_path+'labels/'+(i.split('/',)[-1]).split('.')[0])                             

In [72]:
len(train_mp4_list_output)

186

In [73]:
train_mp4_list_image

['C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/test/image/T004936_006_1_0_1',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/test/image/T004936_020_1_0_1',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/test/image/T004936_022_1_0_1',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/test/image/T004936_033_1_0_1',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/test/image/T000087_000_1_1_1',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/test/image/T000088_000_1_1_1',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/test/image/T000089_000_1_1_1',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/test/image/T000090_000_1_1_1',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/test/image/T000091_000_1_1_1',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/test/image/T000092_000_1_1_1',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/test/image/T000094_000_1_1_1',
 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/test/image/T000095_000_1_1_1',
 'C:/Users/user/Desktop/PM이륜

In [23]:
def bounding_box(points):
    x_coordinates, y_coordinates = zip(*points)
    
    return [[min(x_coordinates), min(y_coordinates), max(x_coordinates), max(y_coordinates)]]

In [29]:
def bbox(anno_info):
    bbox = []

    for obj in anno_info:
        coor = obj['annotation_info']
        if len(coor)==0:
            continue
        elif len(coor) !=1:
#             print(len(coor))
            xyxy = bounding_box(coor)
        else:
            xyxy = coor
            
        b_center_x = (xyxy[0][0] + xyxy[0][2]) / 2
        b_center_y = (xyxy[0][1] + xyxy[0][3]) / 2
        b_width    = xyxy[0][2] - xyxy[0][0]
        b_height   = xyxy[0][3] - xyxy[0][1]
        
        bbox.append("{} {:.6f} {:.6f} {:.6f} {:.6f}".format(obj['category_id'], b_center_x, b_center_y, b_width, b_height))
        
    return bbox

In [44]:
def imwrite(filename, img, params=None):
    try: 
        ext = os.path.splitext(filename)[1]
        result, n = cv2.imencode(ext, img, params)
        if result:
            with open(filename, mode='w+b') as f:
                n.tofile(f)
            return True
        else:
            return False
    except Exception as e:
        print(e)
        return False



In [ ]:
#     x_coordinates, y_coordinates = zip(*points)
# json_data_list[0]['annotations']['anno_info'][0]['frame_annotation'][0]['annotation_info']

In [220]:
# bounding_box(json_data_list[0]['annotations']['anno_info'][0]['frame_annotation'][0]['annotation_info'])

ValueError: too many values to unpack (expected 2)

In [233]:
len(json_data_list[1]['annotations']['anno_info'][1]['frame_annotation'][-2]['annotation_info'])

0

In [254]:
len(json_data_list[1]['annotations']['anno_info'][0]['frame_annotation'][1]['annotation_info'])

1

In [249]:
json_data_list[1]['annotations']['anno_info'][0]['frame_annotation']

[{'annotation_id': '53381',
  'annotation_type': 'bbox',
  'annotation_info': [[1479.9694001534092,
    155.4368305923052,
    1503.9694001534092,
    179.4368305923052]],
  'category_id': '46',
  'category_name': '45_보행자 신호등(적색)_07_신호등_03_도로 시설물'},
 {'annotation_id': '53382',
  'annotation_type': 'bbox',
  'annotation_info': [[955.4251154243976,
    321.6798764592003,
    1020.4251154243976,
    386.6798764592003]],
  'category_id': '53',
  'category_name': '52_자전거_08_PM_04_이동체'},
 {'annotation_id': '53383',
  'annotation_type': 'bbox',
  'annotation_info': [[1787.2024812089728,
    477.4391266407955,
    1919.2024812089728,
    631.4391266407955]],
  'category_id': '53',
  'category_name': '52_자전거_08_PM_04_이동체'},
 {'annotation_id': '53384',
  'annotation_type': 'bbox',
  'annotation_info': [[528.2961978593454,
    462.2958668412755,
    602.2961978593454,
    499.2958668412755]],
  'category_id': '17',
  'category_name': '16_자전거 전용도로_05_노면표시_02_노면유형'},
 {'annotation_id': '53385',
  '

In [217]:
json_data_list[1]['annotations']['anno_info'][209]['frame_annotation']

[{'annotation_id': '53290',
  'annotation_type': 'bbox',
  'annotation_info': [[1488.5333884755141,
    377.0942250814987,
    1592.5333884755141,
    538.0942250814987]],
  'category_id': '124',
  'category_name': '54_오토바이탑승자_위반_09_Rider_04_이동체'},
 {'annotation_id': '53381',
  'annotation_type': 'bbox',
  'annotation_info': [[1477.000415576328,
    147.8545539534717,
    1505.000415576328,
    179.8545539534717]],
  'category_id': '46',
  'category_name': '45_보행자 신호등(적색)_07_신호등_03_도로 시설물'},
 {'annotation_id': '53382',
  'annotation_type': 'bbox',
  'annotation_info': [[958.394100001479,
    324.4820230347132,
    1025.394100001479,
    383.4820230347132]],
  'category_id': '53',
  'category_name': '52_자전거_08_PM_04_이동체'},
 {'annotation_id': '53383',
  'annotation_type': 'bbox',
  'annotation_info': [[1793.197247504204,
    489.1422074000288,
    1918.197247504204,
    613.1422074000288]],
  'category_id': '53',
  'category_name': '52_자전거_08_PM_04_이동체'},
 {'annotation_id': '53384',
  'a

In [213]:
bbox(json_data_list[1]['annotations']['anno_info'][1]['frame_annotation'])

IndexError: list index out of range

In [191]:
print("\n".join(bbox(json_data_list[0]['annotations']['anno_info'][0]['frame_annotation'])), file= open('C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/image/T004936_006_1_0_1_1.txt', "w"))

In [209]:
print("\n".bbox(join(json_data_list[k]['annotations']['anno_info'][i]['frame_annotation'])), file= open('a.txt', "w"))


TypeError: sequence item 0: expected str instance, dict found

In [34]:
cap = cv2.VideoCapture(train_mp4_list[0])
hasFrame, img_frame = cap.read()

In [47]:
imwrite(train_mp4_list_image[0]+'.jpg',img_frame)

True

In [39]:
print('C:/Users/user/Desktop/asd'+'.jpg')

C:/Users/user/Desktop/.jpg


# 영상 이미지화 및 yolo 좌표값 저장

In [76]:
import time



green_color=(0, 255, 0)
red_color=(0, 0, 255)

for v_input, i_output,la_output,k in tqdm(zip(train_mp4_list,train_mp4_list_image,train_mp4_list_labels,range(len(train_mp4_list)))):
    if k==1:
        break;
    check=0
    cap = cv2.VideoCapture(v_input) # <- 앞으로 이 변수를 중심으로 사용할 것
    # Codec은 *'XVID'로 설정. 
    codec = cv2.VideoWriter_fourcc(*'XVID')  # 코덱 객체를 생성
    vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))) #(1920, 1080)
    vid_fps = cap.get(cv2.CAP_PROP_FPS ) 
    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))    
#     global x_coordinates, y_coordinates
#     x_coordinates=[]
#     y_coordinates=[]
    for i in range(5,frame_cnt,6):
        for j in range(6):
            hasFrame, img_frame = cap.read()
            if hasFrame==False:
                check=1
        if check:
            break
#         img_array = np.fromfile(i_output+'_{}'.format(i)+'.jpg', np.uint8)
        imwrite(i_output+'_{}'.format(i)+'.jpg',img_frame)
        print("\n".join(bbox(json_data_list[k]['annotations']['anno_info'][i]['frame_annotation'])), file= open(la_output+'_{}'.format(i)+'.txt', "w"))
    
    
    cap.release()         # 처음에 가져온 이미지. file.close() 와 같은 느낌

1it [00:02,  2.22s/it]


In [67]:
import time



green_color=(0, 255, 0)
red_color=(0, 0, 255)

for v_input, i_output,la_output,k in tqdm(zip(train_mp4_list,train_mp4_list_image,train_mp4_list_labels,range(len(train_mp4_list)))):
    
    cap = cv2.VideoCapture(v_input) # <- 앞으로 이 변수를 중심으로 사용할 것
    # Codec은 *'XVID'로 설정. 
    codec = cv2.VideoWriter_fourcc(*'XVID')  # 코덱 객체를 생성
    vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))) #(1920, 1080)
    vid_fps = cap.get(cv2.CAP_PROP_FPS ) 
    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))    
#     global x_coordinates, y_coordinates
#     x_coordinates=[]
#     y_coordinates=[]
    for i in range(0,frame_cnt):
        hasFrame, img_frame = cap.read()
        #img_array = np.fromfile(i_output+'_{}'.format(i)+'.jpg', np.uint8)
        imwrite(i_output+'_{}'.format(i)+'.jpg',img_frame)
        print("\n".join(bbox(json_data_list[k]['annotations']['anno_info'][i]['frame_annotation'])), file= open(la_output+'_{}'.format(i)+'.txt', "w"))


    cap.release()         # 처음에 가져온 이미지. file.close() 와 같은 느낌

0it [00:00, ?it/s]

2
5
8
11
14
17
20
23
26
29
32
35
38
41
44
47
50
53
56
59
62
65
68
71
74
77
80
83
86
89
92
95
98
101
104
107
110
113
116
119
122
125
128
131
134
137
140
143
146
149
152
155
158
161
164
167
170
173
176


1it [00:03,  3.90s/it]

179


# Annotation 영상에 적용

In [161]:
import time

green_color=(0, 255, 0)
red_color=(0, 0, 255)

for v_input, v_output,k in zip(train_mp4_list,train_mp4_list_output,range(len(train_mp4_list))):
    cap = cv2.VideoCapture(v_input) # <- 앞으로 이 변수를 중심으로 사용할 것
    # Codec은 *'XVID'로 설정. 
    codec = cv2.VideoWriter_fourcc(*'XVID')  # 코덱 객체를 생성

    vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))) #(1920, 1080)
    vid_fps = cap.get(cv2.CAP_PROP_FPS ) 
    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))    
    vid_writer = cv2.VideoWriter(v_output, codec, vid_fps, vid_size) # 처음에는 아무것도 없는 동영상
    for i in tqdm(range(0,frame_cnt)):
        hasFrame, img_frame = cap.read() 
        for j,j_cnt in zip(json_data_list[k]['annotations']['anno_info'][i]['frame_annotation'],range(len(json_data_list[k]['annotations']['anno_info'][i]['frame_annotation']))):
            if j['annotation_type']=='bbox':
                cv2.rectangle(img_frame, xyxy2coco(j['annotation_info']), color=green_color, thickness=2)  # 동영상 프래임 이미지 각각에 박스 넣기
                caption = "frame:{}".format(index)
                cv2.putText(img_frame, caption, caption_bbox(j['annotation_info']),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 1)  # 동영상 프레임 이미지 각각에 몇번째 Frame인지 적기
            elif j['annotation_type']=='polygon':
                points1= np.array(j['annotation_info'],np.int32)
                cv2.polylines(img_frame,[points1],True, color=green_color, thickness=2)  # 동영상 프래임 이미지 각각에 박스 넣기

        vid_writer.write(img_frame) 

    vid_writer.release()  # 나중에 저장한 동영상. file.close() 와 같은 느낌
    cap.release()         # 처음에 가져온 이미지. file.close() 와 같은 느낌

100%|████████████████████████████████████████████████████████████████████████████████| 151/151 [00:03<00:00, 47.29it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 301/301 [00:06<00:00, 44.51it/s]


In [142]:
# xyxy2coco(json_data_list[0]['annotations']['anno_info'][0]['frame_annotation'][1]['annotation_info'])
# xyxy2coco(json_data_list[0]['annotations']['anno_info'][0]['frame_annotation'][0]['annotation_info'])
json_data_list[0]['annotations']['anno_info'][0]['frame_annotation']#2]['annotation_info']

[[964.4173031626094, 204.8604388230037],
 [1046.8456907628824, 221.33497489990324],
 [1343.5878861238662, 257.2794172495022],
 [1382.55403298945, 242.30256627050264],
 [1219.195955745272, 222.83265999780323]]

In [27]:
Video('C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/정제데이터/영상처리/T004935_048_1_0_1_output.mp4') 

In [83]:
json_data_list[0]['annotations']['anno_info'][180]['frame_annotation']

[{'annotation_id': '55192',
  'annotation_type': 'bbox',
  'annotation_info': [[263.70788592590645,
    296.62768936705567,
    300.70788592590645,
    350.62768936705567]],
  'category_id': '124',
  'category_name': '54_오토바이탑승자_위반_09_Rider_04_이동체'},
 {'annotation_id': '55203',
  'annotation_type': 'polygon',
  'annotation_info': [[802.5579238748, 245.29793646630256],
   [804.0483417319499, 282.72351490719643],
   [1163.7441313596337, 332.16367214449997],
   [1337.5930942983919, 260.2747874453021],
   [1156.2506415777907, 237.80951097680276],
   [1031.8587111991965, 222.83265999780323],
   [953.9264174680291, 206.35812392090367],
   [911.9628746897082, 180.89747725660445],
   [877.4928216932301, 180.89747725660445],
   [843.0227686967523, 215.3442345083034]],
  'category_id': '6',
  'category_name': '05_일반 차로_02_차도_01_도로유형'},
 {'annotation_id': '55204',
  'annotation_type': 'polygon',
  'annotation_info': [[965.9077210197593, 203.3710282284101],
   [1048.3361086200323, 219.845564305309

In [82]:
for i in json_data_list[0]['annotations']['anno_info'][180]['frame_annotation']:
    print(i['annotation_info'])
    print("hi")

[[263.70788592590645, 296.62768936705567, 300.70788592590645, 350.62768936705567]]
hi
[[802.5579238748, 245.29793646630256], [804.0483417319499, 282.72351490719643], [1163.7441313596337, 332.16367214449997], [1337.5930942983919, 260.2747874453021], [1156.2506415777907, 237.80951097680276], [1031.8587111991965, 222.83265999780323], [953.9264174680291, 206.35812392090367], [911.9628746897082, 180.89747725660445], [877.4928216932301, 180.89747725660445], [843.0227686967523, 215.3442345083034]]
hi
[[965.9077210197593, 203.3710282284101], [1048.3361086200323, 219.84556430530964], [1345.078303981016, 255.7900066549086], [1384.0444508465998, 240.81315567590903], [1220.6863736024218, 221.34324940320963]]
hi
[[1454.4666945975173, 224.34689410230817], [1168.215384931114, 329.1848509553051], [1258.1372623132302, 348.6547572280045], [1481.4432578121523, 230.33763449390804]]
hi
[[1484.4737741217339, 230.32108548730298], [1258.1703827100744, 353.13126351509936], [1424.5258558669896, 390.581665465896

In [6]:
import torch

In [7]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, custom

# Images
img = 'https://ultralytics.com/images/zidane.jpg'  # or file, Path, PIL, OpenCV, numpy, list

# Inference
results = model(img)

# Results
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.

Using cache found in C:\Users\user/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2021-11-23 torch 1.10.0+cpu CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
C:\Users\user\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
image 1/1: 720x1280 2 persons, 1 tie
Speed: 605.5ms pre-process, 165.4ms inference, 1.0ms NMS per image at shape (1, 3, 384, 640)


In [12]:
a='C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/RAW데이터/대용량데이터/보행안전 도로시설물/01 Bollard_스테인리스/불량/1_01_2_1_01_1_19700101_0000003192.jpg'

In [21]:
video_input_path = rootpath + "1 보행자도로 통행 위반/오토바이/위반/T004935_048_1_0_1.mp4"


In [22]:
result = model(video_input_path)

UnidentifiedImageError: cannot identify image file 'C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/RAW데이터/대용량데이터/PM 사고위험 영상데이터/1 보행자도로 통행 위반/오토바이/위반/T004935_048_1_0_1.mp4'

In [18]:
result .show()

In [43]:
import os
os.chdir("C:/Users/user/Documents/yolov5/")

In [44]:
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [49]:
!cd yolov5


In [53]:
!ls

CONTRIBUTING.md
Dockerfile
LICENSE
README.md
T004935_048_1_0_1.mp4
data
detect.py
export.py
hubconf.py
models
requirements.txt
runs
setup.cfg
train.py
tutorial.ipynb
utils
val.py
yolov5
yolov5s.pt


In [54]:
!pip install -r requirements.txt

In [163]:
!python detect.py --source T004936_006_1_0_1.mp4 --save-txt --save-conf --save-crop

detect: weights=yolov5s.pt, source=T004936_006_1_0_1.mp4, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.0-103-g7a39803 torch 1.10.0+cpu CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
video 1/1 (1/181) C:\Users\user\Documents\yolov5\T004936_006_1_0_1.mp4: 384x640 8 persons, 1 bicycle, 2 cars, 1 skateboard, Done. (0.153s)
video 1/1 (2/181) C:\Users\user\Documents\yolov5\T004936_006_1_0_1.mp4: 384x640 7 persons, 1 bicycle, 2 cars, 1 skateboard, Done. (0.140s)
video 1/1 (3/181) C:\Users\user\Documents\yolov5\T004936_006_1_0_1.mp4: 384x640 8 persons, 1 bicycle, 2 cars, 1 skateboard, Done. (0.119s)
video 1/1 (4/181) C:\Users\user\Documents\yolo

In [ ]:
# usage: detect.py [-h] [--weights WEIGHTS [WEIGHTS ...]] [--source SOURCE] [--imgsz IMGSZ [IMGSZ ...]]
#                  [--conf-thres CONF_THRES] [--iou-thres IOU_THRES] [--max-det MAX_DET] [--device DEVICE] [--view-img]
#                  [--save-txt] [--save-conf] [--save-crop] [--nosave] [--classes CLASSES [CLASSES ...]]
#                  [--agnostic-nms] [--augment] [--visualize] [--update] [--project PROJECT] [--name NAME] [--exist-ok]
#                  [--line-thickness LINE_THICKNESS] [--hide-labels] [--hide-conf] [--half] [--dnn]

In [59]:
isimage = [] 

for (path, dir, files) in os.walk(video_output_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.jpg':
            full=path+'/'+filename
            isimage.append(full)

In [60]:
istxt = [] 
for (path, dir, files) in os.walk(video_output_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt':
            full=path+'/'+filename
            istxt.append(full)

In [61]:
for i,j in zip(isimage,istxt):
    if ((i.split('/')[-1]).split('.')[0])!=((j.split('/')[-1]).split('.')[0]):
        print(i)
        print(j)

In [3]:
txt_path="D:/labels"
image_path="D:/images"

In [4]:
txt_name=[]

In [18]:
image_name=[]


for (path, dir, files) in os.walk(image_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.jpg':
            image_name.append(filename.split('.')[0])

In [19]:
txt_name=[]

for (path, dir, files) in os.walk(txt_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt':
            txt_name.append(filename.split('.')[0])

In [20]:
for i_n,t_n in zip(image_name,txt_name):
    if i_n!=t_n:
        print(i_n+" ",t_n)

In [12]:
def temp_trans(list):
    return [[list[0],list[1]],[list[2],list[1]],[list[2],list[3]],[list[0],list[3]]]

In [25]:
def bbox(list):
    bbox = []
  
    b_center_x = (list[0] + list[2]) / 2
    b_center_y = (list[1] + list[3]) / 2
    b_width    = list[2] - list[0]
    b_height   = list[3] - list[1]
        
    bbox.append("{:.6f}, {:.6f}, {:.6f}, {:.6f}".format(b_center_x, b_center_y, b_width, b_height))
        
    return bbox

In [18]:
def yolotobbox(list):
    x,y,w,h = [list[i] for i in range(4)]
    x1= x-w/2
    x2=x+w/2
    y1=y-h/2
    y2=y+h/2
    
    return [[x1,y1],[x2,y1],[x2,y2],[x1,y2]]

In [29]:
from shapely.geometry import Polygon

def check_intersection(polygon_coor,bbox_coor):
    polygon1 = Polygon(polygon_coor)
    bbox1 = Polygon(yolotobbox(bbox_coor))
        
#         [[852.9553951634875, 298.36530925266],
#            [947.4197928492978, 302.86080307628106],
#            [1071.8728882131438, 299.86380719386705],
#            [1202.323723112596, 293.869815429039],
#            [1280.2943370754874, 281.8818318993828],
#            [1301.286425450112, 278.88483601696873],
#            [1293.286016035324, 399.2675918554472],
#            [1281.7937719593895, 515.6475107276779],
#            [1268.2988580042734, 616.046872788548],
#            [1242.8084649779435, 729.9327163202814],
#            [1044.8830603029119, 638.5243419066534],
#            [834.9621765566665, 599.5633954352708],
#            [619.0435532748138, 553.1099592578531],
#            [490.0921532592629, 524.6384983749198]])

#     polygon2 = Polygon( temp_trans([788.9367069939269,   #yolotobbox 로 변경
#             327.77343261579534,
#             865.9367069939269,
#             494.77343261579534]))

    intersect = polygon1.intersection(polygon2).area
    mainarea = polygon2.area
    iom = intersect / mainarea
    
    return iom

#0.9931090119187697

In [ ]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
 

createFolder('D:/annotation_test')


